In [1258]:
from cumulants import compute_cumulants, compute_compressed_cumulants

In [1259]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1260]:
import torch

In [1261]:
L = torch.Tensor([0.94981112, 0.94981112, 0.99568268, 0.99568268])
R = torch.Tensor([[1.24742243, 0.34459052, 0.77200966, 0.57715386],
       [0.34459052, 1.24742243, 0.57715386, 0.77200966],
       [0.38600483, 0.28857693, 1.50352339, 0.76307132],
       [0.28857693, 0.38600483, 0.76307132, 1.50352339]])
mux = torch.Tensor([0., 0.])

In [1262]:
C, K = compute_cumulants(R, L, mux, return_full_K=True)

In [1263]:
from tick.hawkes.inference.base import LearnerHawkesNoParam
from tick.hawkes import HawkesCumulantMatching

In [1264]:
integration_support = 1 # H 
nphc = HawkesCumulantMatching(integration_support, cs_ratio=.15, tol=1e-10, step=0.3)

# simulation

In [1089]:
from simulate import Hawkes_Shot_Noise_Simulate as HSNS

In [1091]:
import numpy as np

In [1092]:
dic = {'alpha': np.array([[0.0, 0.15, 0.4 , 0. ],
                            [0.15, 0.0, 0. , 0.4 ],
                            [0.2 , 0.0, 0.0, 0.4 ],
                            [0.0, 0.2 , 0.4 , 0.0]]),
 'beta': 100,
 'endo_baseline': np.array([0.1 , 0.1 , 0.25, 0.25]),
 'exo_baseline': np.array([0.2, 0.2]),
 'delay': [0.001, 0.001, 0.001, 0.001]}


In [1097]:
simu = HSNS(dim_endo=4,
            dim_exo=2,
            end_time=10000, 
            verbose=False)

In [1098]:
simu.set_parameters(**dic)

[0.001, 0.001, 0.001, 0.001]


In [1099]:
ts, labels = simu.simulate(n_realization=10)

Simulating...:   0%|          | 0/10 [00:00<?, ?it/s]

In [1100]:
simu.compute_cumulants()

In [1101]:
simu.L

array([0.8372093 , 0.8372093 , 1.02906977, 1.02906977])

In [1102]:
simu.C

tensor([[1.9534, 1.0864, 1.7603, 1.3135],
        [1.0864, 1.9534, 1.3135, 1.7603],
        [1.7603, 1.3135, 2.6025, 1.9675],
        [1.3135, 1.7603, 1.9675, 2.6025]])

In [1103]:
simu.K

tensor([[ 7.2068,  4.1074,  7.6031,  5.9952],
        [ 4.1074,  7.2068,  5.9952,  7.6031],
        [ 6.5806,  5.0734, 11.5731,  9.3093],
        [ 5.0734,  6.5806,  9.3093, 11.5731]])

In [1104]:
len(ts)

10

# Estimation

In [1146]:
from estimate import Hawkes_Shot_Noise_Estimate

## sparse


In [1165]:
from learners import sparse_phi

In [1166]:
hsne = sparse_phi(4,2)

In [1167]:
hsne.set_data(ts, end_time=10000)

In [1168]:
hsne.estimate_cumulants(H=1)

In [1169]:
hsne.L_emp

array([0.83925, 0.8388 , 1.03526, 1.03353])

In [1170]:
hsne.set_init_values()
hsne.set_variables()

/mnt/nfs/nrdata02-users-data/ruan/ANotebook/A6-HawkesShotNoise/new/learners.py:187: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  requires_grad=True,


In [1171]:
hsne.fit(20000, 0.001, 1e-8, 1000)

     n_iter  |  objective  |    rel_obj  
-----------------------------------------
          0  |   4.18e+02  |   7.32e-02
       1000  |    7.9e-01  |   1.39e-03
       2000  |   2.72e-01  |   4.76e-04
       3000  |   2.07e-01  |   2.44e-04
       4000  |   1.54e-01  |   3.57e-04
       5000  |   1.02e-01  |   4.63e-04
       6000  |   6.24e-02  |   4.97e-04
       7000  |   3.89e-02  |   4.47e-04
       8000  |   2.48e-02  |   4.70e-04
       9000  |   1.51e-02  |   4.95e-04
      10000  |   9.90e-03  |   2.91e-04
      11000  |   8.42e-03  |   6.53e-05
      11587  |   8.29e-03  |     0.e+00
      20000  |   8.29e-03  |     0.e+00


In [1172]:
hsne.init_phi, hsne.init_exo_baseline

(tensor([0.1528, 0.4098, 0.4451, 0.4047]), array([0.]))

In [1173]:
hsne.variables

[tensor([0.1510, 0.3940, 0.2048, 0.3981], requires_grad=True),
 tensor([0.2099], requires_grad=True)]

In [1174]:
hsne.adjacency

array([[0.        , 0.1510089 , 0.39403957, 0.        ],
       [0.1510089 , 0.        , 0.        , 0.39403957],
       [0.20476322, 0.        , 0.        , 0.39812493],
       [0.        , 0.20476322, 0.39812493, 0.        ]], dtype=float32)

In [1175]:
hsne.endo_baseline

array([0.09475954, 0.09492324, 0.24204753, 0.23972099], dtype=float32)

In [1176]:
dic

{'alpha': array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]]),
 'beta': 100,
 'endo_baseline': array([0.1 , 0.1 , 0.25, 0.25]),
 'exo_baseline': array([0.2, 0.2]),
 'delay': [0.001, 0.001, 0.001, 0.001]}

In [1177]:
hsne.objective(var_phi = torch.tensor([0.15,0.4,0.2,0.4]), var_exo_mu=torch.tensor([0.2]))

tensor(0.0102)

In [1178]:
hsne.objective()

tensor(0.0083, grad_fn=<AddBackward0>)

## full


In [1193]:
from learners import general_phi

In [1200]:
hsne = general_phi(4,2)

In [1201]:
hsne.set_data(ts, end_time=10000)

In [1202]:
hsne.estimate_cumulants(H=1)

In [1203]:
hsne.L_emp

array([0.83925, 0.8388 , 1.03526, 1.03353])

In [1204]:
hsne.set_init_values()
hsne.set_variables()

In [1205]:
hsne.fit(20000, 0.001, 1e-8, 1000)

     n_iter  |  objective  |    rel_obj  
-----------------------------------------
          0  |   2.54e+00  |   1.31e+00
       1000  |   1.63e-01  |   2.02e-04
       2000  |   1.20e-01  |   4.98e-04
       3000  |   4.30e-02  |   1.01e-03
       4000  |   2.01e-02  |   6.07e-04
       5000  |   1.39e-02  |   2.83e-04
       6000  |   1.07e-02  |   2.96e-04
       7000  |   7.41e-03  |   4.60e-04
       8000  |   5.47e-03  |   4.14e-04
       9000  |   3.24e-03  |   6.46e-04
      10000  |   1.73e-03  |   4.14e-03
      11000  |   1.11e-03  |   3.21e-02
      12000  |   6.92e-04  |    3.2e-04
      13000  |   5.35e-04  |   4.76e-03
      14000  |   4.26e-04  |   7.57e-03
      15000  |   4.42e-04  |   8.74e-02
      16000  |   3.20e-04  |   9.45e-06
      17000  |    2.9e-04  |   5.80e-05
      18000  |   2.69e-04  |    4.3e-04
      19000  |   2.52e-04  |   9.82e-04
      20000  |   2.48e-04  |   1.71e-02


In [1206]:
hsne.init_phi, hsne.init_exo_baseline

(array([[-4.23908883e-04,  1.52765646e-01,  4.09790254e-01,
         -1.64271538e-02],
        [ 1.52724685e-01, -1.39251464e-03, -1.26834153e-02,
          4.01025559e-01],
        [ 4.55135454e-01, -1.40906724e-02, -1.31235742e-01,
          4.05018146e-01],
        [-1.82296442e-02,  4.45147958e-01,  4.04679596e-01,
         -1.20437913e-01]]),
 array([0., 0.]))

In [1207]:
hsne.variables

[tensor([[ 0.0008,  0.1808,  0.3696,  0.0318],
         [ 0.1121,  0.0239, -0.0134,  0.3717],
         [ 0.2414,  0.0680, -0.0276,  0.3724],
         [-0.0582,  0.2203,  0.4190, -0.0004]], requires_grad=True),
 tensor([0.1826, 0.2291], requires_grad=True)]

In [1208]:
hsne.adjacency

array([[ 0.00076303,  0.18075623,  0.36959124,  0.03176131],
       [ 0.11213133,  0.02387973, -0.01338104,  0.37173685],
       [ 0.241442  ,  0.06795167, -0.02764461,  0.3723709 ],
       [-0.05821165,  0.22028972,  0.41898373, -0.00041969]],
      dtype=float32)

In [1209]:
hsne.endo_baseline

array([0.08894531, 0.12517518, 0.23679812, 0.23514187], dtype=float32)

In [1210]:
dic

{'alpha': array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]]),
 'beta': 100,
 'endo_baseline': array([0.1 , 0.1 , 0.25, 0.25]),
 'exo_baseline': array([0.2, 0.2]),
 'delay': [0.001, 0.001, 0.001, 0.001]}

In [1214]:
simu.adjacencys

[array([[0.  , 0.15, 0.4 , 0.  ],
        [0.15, 0.  , 0.  , 0.4 ],
        [0.2 , 0.  , 0.  , 0.4 ],
        [0.  , 0.2 , 0.4 , 0.  ]])]

In [1218]:
hsne.objective(var_phi=torch.tensor(simu.adjacencys[0], dtype=torch.float32), 
               var_exo_mu=torch.tensor(simu.exo_baselines, dtype=torch.float32))

tensor(0.0102)

In [1219]:
hsne.objective()

tensor(0.0002, grad_fn=<AddBackward0>)

# Sparse P

In [1236]:
from learners import sparse_phi_compress

In [1237]:
hsne = sparse_phi_compress(4,2)

In [1238]:
hsne.set_data(ts, end_time=10000)

In [1239]:
hsne.estimate_cumulants(H=1)

In [1240]:
hsne.L_emp

array([0.83925, 0.8388 , 1.03526, 1.03353])

In [1241]:
hsne.C_emp, hsne.K_emp

(array([[1.9736101 , 1.08571816, 1.78761121, 1.32916633],
        [1.08571816, 1.94094919, 1.32516642, 1.75931363],
        [1.78761121, 1.32516642, 2.6297321 , 1.99379405],
        [1.32916633, 1.75931363, 1.99379405, 2.62299861]]),
 array([[ 7.3408479 ,  4.06223836,  7.79410144,  6.04563778],
        [ 4.10167206,  7.03270722,  6.01206981,  7.56048973],
        [ 6.76476087,  5.04184983, 11.71713284,  9.36170254],
        [ 5.16040344,  6.48012291,  9.36025347, 11.6775938 ]]))

In [1255]:
hsne.set_init_values()
hsne.set_variables()

/mnt/nfs/nrdata02-users-data/ruan/ANotebook/A6-HawkesShotNoise/new/learners.py:324: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_phi[0, 1],


In [1256]:
simu.C, simu.K

(tensor([[1.9534, 1.0864, 1.7603, 1.3135],
         [1.0864, 1.9534, 1.3135, 1.7603],
         [1.7603, 1.3135, 2.6025, 1.9675],
         [1.3135, 1.7603, 1.9675, 2.6025]]),
 tensor([[ 7.2068,  4.1074,  7.6031,  5.9952],
         [ 4.1074,  7.2068,  5.9952,  7.6031],
         [ 6.5806,  5.0734, 11.5731,  9.3093],
         [ 5.0734,  6.5806,  9.3093, 11.5731]]))

In [1269]:
C, K = compute_cumulants(simu.R, simu.L, simu.exo_baselines, return_full_K=True)

In [1283]:
K = K.round()

In [1284]:
C, K

(tensor([[1.9534, 1.0864, 1.7603, 1.3135],
         [1.0864, 1.9534, 1.3135, 1.7603],
         [1.7603, 1.3135, 2.6025, 1.9675],
         [1.3135, 1.7603, 1.9675, 2.6025]]),
 tensor([[[ 7.,  4.,  7.,  5.],
          [ 4.,  4.,  4.,  4.],
          [ 7.,  4.,  8.,  6.],
          [ 5.,  4.,  6.,  6.]],
 
         [[ 4.,  4.,  4.,  4.],
          [ 4.,  7.,  5.,  7.],
          [ 4.,  5.,  6.,  6.],
          [ 4.,  7.,  6.,  8.]],
 
         [[ 7.,  4.,  8.,  6.],
          [ 4.,  5.,  6.,  6.],
          [ 8.,  6., 12.,  9.],
          [ 6.,  6.,  9.,  9.]],
 
         [[ 5.,  4.,  6.,  6.],
          [ 4.,  7.,  6.,  8.],
          [ 6.,  6.,  9.,  9.],
          [ 6.,  8.,  9., 12.]]]))

In [1285]:
eyes_2 = torch.Tensor([[1, -1, 0, 0], [0, 0, 1, -1]])
C_2 = torch.matmul(torch.matmul(eyes_2, C), torch.transpose(eyes_2, 0, 1))


In [1293]:
torch.matmul(torch.matmul(eyes_2, K[2]), torch.transpose(eyes_2, 0, 1))

tensor([[4., 2.],
        [2., 3.]])

In [1286]:
C_2

tensor([[1.7340, 0.8935],
        [0.8935, 1.2700]])

In [1287]:
K_2 = torch.matmul(torch.matmul(eyes_2, K), torch.transpose(eyes_2, 0, 1))

In [1288]:
K_2.size()

torch.Size([4, 2, 2])

In [1289]:
K_2

tensor([[[3., 2.],
         [2., 2.]],

        [[3., 2.],
         [2., 2.]],

        [[4., 2.],
         [2., 3.]],

        [[4., 2.],
         [2., 3.]]])

In [1277]:
K_2 = torch.transpose(torch.matmul(eyes_2, torch.transpose(K_2, 0, 1)), 0, 1)

In [1278]:
K_2

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]])

In [1270]:
compute_compressed_cumulants(C, K)

(tensor([[1.7340, 0.8935],
         [0.8935, 1.2700]]),
 tensor([[[0., 0.],
          [0., 0.]],
 
         [[0., 0.],
          [0., 0.]]]))